In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 🔧 AGI Voice Agent - Language Model Fine-Tuning (Notebook 7)

### Objective:
Fine-tune an LLM (e.g., GPT-2, LLaMA, Falcon) on:
- Custom AGI reasoning datasets
- Improve reasoning, dialogue, and retrieval capabilities



Requirements:
- Hugging Face Transformers
- PyTorch
- Datasets library




📌 1. Install Libraries

!pip install transformers datasets torch




📌 2. Load Libraries

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch
import pandas as pd




📌 3. Load Custom Dataset

Assuming the AI Reasoning dataset with question and answer:

df = pd.read_parquet('processed/ai_reasoning.parquet')
df = df[['question', 'answer']].dropna()

df['text'] = df['question'] + ' Answer: ' + df['answer']

dataset = Dataset.from_pandas(df[['text']])
dataset = dataset.train_test_split(test_size=0.1)
dataset




📌 4. Tokenizer and Model

model_name = "gpt2"  # Replace with 'meta-llama/Llama-2-7b-hf' for LLaMA if available

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)



📌 5. Training Arguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=2,
    push_to_hub=False,
)




📌 6. Initialize Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)




📌 7. Start Fine-Tuning

trainer.train()




📌 8. Save Fine-Tuned Model

model.save_pretrained("./fine_tuned_agi_model")
tokenizer.save_pretrained("./fine_tuned_agi_model")




📌 9. Inference Example

input_text = "What is the purpose of consciousness?"
inputs = tokenizer.encode(input_text, return_tensors='pt')
outputs = model.generate(inputs, max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

SyntaxError: invalid character '📌' (U+1F4CC) (148719680.py, line 18)